In [1]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name= "llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

# another llama API key = "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"
# model name = 'meta-llama/llama-4-scout-17b-16e-instruct'

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [20]:
# pip install langchain_community

In [21]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/senior-technical-product-manager/job/R-60270")
page_data = loader.load().pop().page_content
print(page_data)






















Senior Technical Product Manager










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous M

In [22]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [23]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Technical Product Manager',
 'experience': '2 years',
 'skills': ['SAP',
  'Billing and price domain',
  'System Configuration in FI or SD modules',
  'Jira',
  'Quality Center',
  'FICO and legacy systems',
  'Change request processes',
  'Agile methodology'],
 'description': "Design, develop and manage activities for a specific product or group of products from product definition and planning through production, release, and end of life; analysis, design, implementation, operations, and support of the organization's IT resources, systems, and software/database products; develop information technology strategies, polices and plans; formulate product strategies and identify new product opportunities; develop project plans designed to address customer needs and expand market share; integrate and communicate moderately complex information to audiences who are not familiar with the subject matter; utilize knowledge, skills, and abilities to identify areas of opportunity, 

In [24]:
type(json_res)

dict

In [25]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

Techstack  \
0                React, Node.js, MongoDB   
1               Angular,.NET, SQL Server   
2      Vue.js, Ruby on Rails, PostgreSQL   
3                  Python, Django, MySQL   
4              Java, Spring Boot, Oracle   
5             Flutter, Firebase, GraphQL   
6                  WordPress, PHP, MySQL   
7                    Magento, PHP, MySQL   
8         React Native, Node.js, MongoDB   
9                  iOS, Swift, Core Data   
10       Android, Java, Room Persistence   
11             Kotlin, Android, Firebase   
12       Android TV, Kotlin, Android NDK   
13                     iOS, Swift, ARKit   
14        Cross-platform, Xamarin, Azure   
15          Backend, Kotlin, Spring Boot   
16         Frontend, TypeScript, Angular   
17    Full-stack, JavaScript, Express.js   
18  Machine Learning, Python, TensorFlow   
19               DevOps, Jenkins, Docker   

                                                Links  
0                 https://example.com/react-portfolio  
1               https://example.com/angular-portfolio  
2                   https://example.com/vue-portfolio  
3                https://example.com/python-portfolio  
4                  https://example.com/java-portfolio  
5               https://example.com/flutter-portfolio  
6             https://example.com/wordpress-portfolio  
7               https://example.com/magento-portfolio  
8          https://example.com/react-native-portfolio  
9                   https://example.com/ios-portfolio  
10              https://example.com/android-portfolio  
11       https://example.com/kotlin-android-portfolio  
12           https://example.com/android-tv-portfolio  
13               https://example.com/ios-ar-portfolio  
14              https://example.com/xamarin-portfolio  
15       https://example.com/kotlin-backend-portfolio  
16  https://example.com/typescript-frontend-portfolio  
17        https://example.com/full-stack-js-portfolio  
18            https://example.com/ml-python-portfolio  
19               https://example.com/devops-portfolio

In [26]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [27]:
job = json_res
job['skills']

['SAP',
 'Billing and price domain',
 'System Configuration in FI or SD modules',
 'Jira',
 'Quality Center',
 'FICO and legacy systems',
 'Change request processes',
 'Agile methodology']

In [28]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/java-portfolio'}]]

In [29]:
job

{'role': 'Senior Technical Product Manager',
 'experience': '2 years',
 'skills': ['SAP',
  'Billing and price domain',
  'System Configuration in FI or SD modules',
  'Jira',
  'Quality Center',
  'FICO and legacy systems',
  'Change request processes',
  'Agile methodology'],
 'description': "Design, develop and manage activities for a specific product or group of products from product definition and planning through production, release, and end of life; analysis, design, implementation, operations, and support of the organization's IT resources, systems, and software/database products; develop information technology strategies, polices and plans; formulate product strategies and identify new product opportunities; develop project plans designed to address customer needs and expand market share; integrate and communicate moderately complex information to audiences who are not familiar with the subject matter; utilize knowledge, skills, and abilities to identify areas of opportunity, 

In [30]:
# prompt_email = PromptTemplate.from_template(
#         """
#         ### JOB DESCRIPTION:
#         {job_description}
        
#         ### INSTRUCTION:
#         You are Israel Afolabi, a business development executive at EasyTech. EasyTech is an AI & Software Consulting company dedicated to facilitating
#         the seamless integration of business processes through automated tools. 
#         Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
#         process optimization, cost reduction, and heightened overall efficiency. 
#         Your job is to write a cold email to the client regarding the job mentioned above describing the capability of EasyTech 
#         in fulfilling their needs.
#         Also add the most relevant ones from the following links to showcase EasyTech's portfolio: {link_list}
#         Remember you are Israel. 
#         Do not provide a preamble.
#         ### EMAIL (NO PREAMBLE):
        
#         """
#         )

# chain_email = prompt_email | llm
# res = chain_email.invoke({"job_description": str(job), "link_list": links})
# print(res.content)

In [31]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Israel Afolabi, a who just completed your training and worked on several projects, now his seeking a role that alligns with his project and 
        sinificantly want to contribute to organizations growth
        # Draft a compelning cold mail
        Also add the most relevant ones from the following links to showcase EasyTech's portfolio: {link_list}
        Remember you are Israel. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Senior Technical Product Manager Role at Nike

Dear Hiring Manager,

I am excited to apply for the Senior Technical Product Manager position at Nike, as advertised. With my strong background in IT and product management, I am confident that I can make a significant contribution to the growth and success of your organization.

As a highly motivated and experienced professional, I have a deep understanding of the skills required for this role, including SAP, Billing and Price domain, System Configuration in FI or SD modules, Jira, Quality Center, FICO and legacy systems, Change request processes, and Agile methodology. My experience in designing, developing, and managing activities for specific products or groups of products from product definition and planning through production, release, and end of life will be a valuable asset to your team.

I would like to highlight my proficiency in analyzing complex problems, identifying areas of opportunity, and navigating